## Setup

In [1]:
import json, openai
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain.agents import Tool, tool, load_tools, initialize_agent, AgentType, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from langchain.tools import BaseTool, StructuredTool
from langchain.utilities.google_serper import GoogleSerperAPIWrapper
from langchain.docstore import Wikipedia
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools.render import render_text_description, format_tool_to_openai_function
from langchain.schema import HumanMessage, AIMessage
from langchain.schema.agent import AgentFinish, AgentAction
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:
load_dotenv()
gpt3_5 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

## Tools

The tools are functionality that agents can use to perform their tasks.

> Note: Only LLMs that support function calling can use tools from a function. At this time, OpenAI provides the most support.

> TODO: Demonstrate using a local LLM with tool

In [3]:
# langchain has some easy Tool wrappers for some tools
google = GoogleSerperAPIWrapper()
google_tool = Tool.from_function(
  func=google.run,
  name="Google",
  description="useful for when you need search for something you are not sure about"
)

In [4]:
# can simply invoke
google_tool.invoke("how to make a cake")

'Ingredients ; 1 cup white sugar ; ½ cup unsalted butter ; 2 large eggs ; 2 teaspoons vanilla extract ; 1 ½ cups all-purpose flour. How to make vanilla cake. 1. Whip the eggs and sugar – Beat the eggs ... How to make vanilla cake. 2. Gradually add flour – Whisk together the ... cake in 2 minutes! you will make this cake every day! easy and quick to prepare. very ... Duration: 3:37. Posted: Jul 17, 2022. Easy to adjust Vanilla Cake to make vanilla cupcakes, birthday cake, vanilla sheet cakes and ... Duration: 5:12. Posted: Sep 21, 2023. How to Bake a Cake · Step 13: Frost and Decorate · Step 12: Add a Crumb Coat · Step 11: Assemble the Cake · Step 10: Cool the Cake Layers · Step 9: Check Cake ... Ingredients ; cooking spray ; 2 ⅔ · all-purpose flour, or more as needed ; 1 · white sugar ; 1 · baking powder ; 1 · vanilla extract. A classic vanilla cake recipe is often simple and beginner-friendly. Combine flour, sugar, butter, eggs, milk, and vanilla extract. Make the cake: Whisk the cake

In [ ]:
# using decorator
@tool
def get_word_length(word: str) -> int:
  """Returns the length of a word."""
  return len(word)

In [ ]:
# support for multi input data structures
class Word(BaseModel):
  word: str = Field()
  strategy: str = Field()
  
def word_length(word: str, strategy: str) -> int:
  return len(word) if strategy == "normal" else len(word) * 2

word_tool = StructuredTool.from_function(
  func=word_length,
  name="Word_Length_Calculator",
  description="useful to calculate the length of a word. use strategy='normal' for normal length, strategy='turbo' for turbo length.",
  args_schema=Word
)

In [ ]:
class CoolNameTool(BaseTool):
  name = "cool_name"
  description = "useful to determine if a name is cool"
  
  def _run(self, query: str) -> str:
    return "Yes, this is a cool name" if query == "Brian" else "No, this is not a cool name"

In [ ]:
tools = [
  Tool(
    name="Search",
    func=google.run,
    description="useful for when you need information about current events and data"
  ),
  word_tool
]

In [ ]:
render_text_description(tools)

## Provided Agents

Every agent type in `langchain` has different characteristics, but they mainly differ in what prompt they are using and how they determine what tools to use. It will use either `ReAct` from langchain or `OpenAI` to manage tool invocations. We will first look at the available `off-the-shelf agent` options.

### Zero-shot ReAct
Uses the ReAct framework to determine which tool to use based solely on the tool's description. A very general purpose action agent. ONLY supports tools with single string input.

In [ ]:
# can load builtin tools from langchain.agents
tools = load_tools(["llm-math"], gpt3_5)
# will create simple AgentExecutor (no prompt or pipeline)
agent = initialize_agent(tools, gpt3_5, AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
print(agent.agent.llm_chain.prompt.template)
agent.run("What is 25 to the power of 0.43 power?")

### Structured Input ReAct (Structured Chat)

Just like zero-shot but supports multi-input tools.

In [ ]:
tools = [word_tool]
agent = initialize_agent(tools, gpt3_5, AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
print(agent.agent.llm_chain.prompt.format(agent_scratchpad="{agent_scratchpad}", input="{input}"))
agent.run("What is the length of the word 'boulder'?")

### OpenAI

This agent will let OpenAI make the decision on what tool to use. The tool can accept one or more inputs. You will notice the prompt is very basic. The tool options along with their inputs will be passed along to OpenAI. This will invoke one tool at a time per response. However, the `AgentType.OPENAI_MULTI_FUNCTIONS` will allow a list of tool invocations to be processed.

In [ ]:
tools = [google_tool]
agent = initialize_agent(tools, gpt3_5, AgentType.OPENAI_FUNCTIONS, verbose=True)
print(agent.agent.prompt.format(agent_scratchpad=[], input="{input}"))
agent.run("What is the highest priced stock?")

### Conversational

This agent is similar to other ReAct agents, but this one has a system prompt optimized for conversations. I have included memory, since that is common with coversations.

In [ ]:
tools = [CoolNameTool()]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = initialize_agent(tools, gpt3_5, AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)
print(agent.agent.llm_chain.prompt.format(agent_scratchpad=[HumanMessage(content="this is the scratchpad")], chat_history=[HumanMessage(content="this is the chat history")], input="{input}"))
agent.run("hello")
agent.run("My name is Brian. Is my name cool?")

In [ ]:
# you can see the history of the conversation from the memory
memory.load_memory_variables({})

### Self-ask with Search

A specialized agent to be used with a search tool. The LLM must not be a chat model but a normal model. The search tool name must be `Intermediate Answer`.

In [ ]:
# search tool name must be "Intermediate Answer"
llm = OpenAI(temperature=0)
tools = [Tool.from_function(func=google.run, name="Intermediate Answer", description="useful for when you need to ask with search")]
agent = initialize_agent(tools, llm, AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
print(agent.agent.llm_chain.prompt.template)
agent.run("What is the highest grossing movie of all time?")

### ReAct Document Store

Uses Wikipedia to search and retrieve information. Requires the `wikipedia` python package.

In [ ]:
llm = OpenAI(temperature=0)
docstore = DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="useful for when you need to ask with search"
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="useful for when you need to ask with lookup"
  )
]
agent = initialize_agent(tools, llm, AgentType.REACT_DOCSTORE, verbose=True)
print(agent.agent.llm_chain.prompt.template)
agent.run("Who is the youngest US president?")

## Custom Agent

The agent runtime provided by LangChain is `AgentExecutor`. It does support others runtimes like `Baby AGI` and `Auto GPT`. Up to this point, we have been using predefined agents. Now we will turn to creating our own agents.

1. Building an agent starts with a LLM:

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm.invoke("Who are you?")

2. Then we have tools, which is the same as we have seen before, but let's define one again:

In [ ]:
@tool
def is_number_even(number: int) -> bool:
  """Returns true if number is even."""
  return number % 2 == 0

tools = [is_number_even]

3. Next, we have the prompt. When using OpenAI the prompt is simple, but if using another API the prompt may need more instructions and examples. Let's create a prompt for OpenAI:

In [ ]:
prompt = ChatPromptTemplate.from_messages([
  (
    "system",
    "You are a helpful AI assistant."
  ),
  ("user", "{input}"),
  MessagesPlaceholder(variable_name="agent_scratchpad")
])
print(prompt.format(input="{input}", agent_scratchpad=[]))

4. The next step is to make the LLM aware of the available tools.

In [ ]:
openai_tools = [format_tool_to_openai_function(t) for t in tools]
llm_with_tools = llm.bind(functions=openai_tools)
print(json.dumps(openai_tools[0], indent=2))

5. Then we need to create the chain using the variables from the previous steps to create a `RunnableSequence`:

In [ ]:
chain = (
  {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"])
  }
  | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()
)
print(type(chain))

In [ ]:
# take a look at the AgentAction response from using the chain
chain.invoke({"input": "Is 4 an even number?", "intermediate_steps": []})

6. Right now we just have a runnable sequence (i.e., chain), but the chain still doesn't have any dynamic capability. There are multiple ways to do this. First, we'll look at the hard way which demonstrates the process, but keep in mind this can be simplified:

In [ ]:
user_input = "Is 343565 an even number?"
intermediate_steps = []
while True:
  output: AgentAction = chain.invoke({"input": user_input, "intermediate_steps": intermediate_steps})
  if isinstance(output, AgentFinish):
    final_result = output.return_values["output"]
    break # we have the final answer
  else:
    tool: Tool = {"is_number_even": is_number_even}[output.tool] # we could have simply called the tool directly
    tool_result = tool.run(output.tool_input) # AgentAction knows the tool input
    intermediate_steps.append((output, tool_result))

print(final_result)

We can simply this process by using the `AgentExecutor` (the runtime):

In [ ]:
agent = AgentExecutor(agent=chain, tools=tools, verbose=True) # similar to initialize_agent but we provide the custom agent
agent.invoke({"input": "Is 4 an even number?"})

7. (Optional) We can also give the agent memory if we want a more conversational approach. This requires us to change the `prompt` and `agent`:

In [ ]:
memory_key = "chat_history"
prompt = ChatPromptTemplate.from_messages([
  (
    "system",
    "You are a helpful AI assistant."
  ),
  MessagesPlaceholder(variable_name=memory_key), # this is needed to hold the history of the conversation
  ("user", "{input}"),
  MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
chain = (
  {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"]),
    "chat_history": lambda x: x["chat_history"]
  }
  | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()
)

In [ ]:
agent = AgentExecutor(agent=chain, tools=tools, verbose=True)

In [ ]:
chat_history = []
input1 = "is 34234 an even number?"
input2 = "is 1 an even number?"
result1 = agent.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend([
  HumanMessage(content=input1),
  AIMessage(content=result1["output"])
])
result2 = agent.invoke({"input": input2, "chat_history": chat_history})
chat_history.extend([
  HumanMessage(content=input2),
  AIMessage(content=result2["output"])
])

In [ ]:
for c in chat_history:
  print(type(c), c.content)

### OpenAI Assistants

OpenAI assistants are agents and can have tools of its own, which then can be in turn combined with functional tool defined.

In [ ]:
translator_assitant = OpenAIAssistantRunnable.create_assistant(
  name="language translator",
  instructions="You are a language translator. Translate the user text from English into whatever language is requsted",
  tools=[],
  model="gpt-3.5-turbo",
)
output = translator_assitant.invoke({
  "content": "'Suck on my dongle' to Spanish"
})

In [57]:
assistant_id = translator_assitant.assistant_id
print(assistant_id)
output[0].content[0].text.value # ThreadMessage: very nested structure

asst_bYKqzdRLtpe5R19FSvM1zHKd


'"Chúpame el dongle" in Spanish.'

In [ ]:
translator_assitant.as_agent = True
agent = AgentExecutor(agent=translator_assitant, tools=[]) # can be combined with local tools
output = agent.invoke({
  "content": "'Suck on my dongle' to German"
})

In [66]:
output["output"] # basic dict

{'content': "'Suck on my dongle' to German", 'output': '"Leck meinen Dongle" in German.', 'thread_id': 'thread_I1CAzbl9hPnYqLENnKwYeD85', 'run_id': 'run_AAzDYuz8KHsFtxHKtl6kaKAu'}


'"Leck meinen Dongle" in German.'

You can also use an existing assistant as an agent with LangChain:

In [ ]:
existing_assistant = OpenAIAssistantRunnable(assistant_id=assistant_id, as_agent=True)
output = existing_assistant.invoke({ "content": "'hello' to Spanish" })

In [68]:
print(type(output))
output.return_values["output"] # child of AgentAction

<class 'langchain.agents.openai_assistant.base.OpenAIAssistantFinish'>


"'hola'"

In [ ]:
openai.Client().beta.assistants.delete(assistant_id)